# ESSE 2220 – Lab 1 Report

> Replace all bracketed ALL-CAPS placeholders with your own content. Remove instructional notes once complete.

**Course:** ESSE 2220  
**Lab Title:** Raspberry Pi LED Morse Code  
**Date Performed:** [DATE]  
**Date Submitted:** [DATE]  
**Group Number:** [GROUP #]  
**Members:** [NAME 1] · [NAME 2] · [NAME 3] · [NAME 4]

---

## 1. Names & Group Info
Provide group number, member names (as they appear on the roster), and the lab date.

---

## 2. Circuit Setup
**GPIO Pin Used for LED:** `[e.g., GPIO 17 (BOARD pin 11)]`

Short description (in your own words):  
“We connected the LED to pin [PIN] and ground so the Raspberry Pi could control it through Python.”

**Schematic / Wiring Notes:**  
- Anode (long leg) → Series resistor (e.g., 220 Ω) → GPIO pin.  
- Cathode (short leg) → Ground (GND).  
- Using BCM vs BOARD numbering? State which you used and why.

> (Optional) Add a simple ASCII diagram or insert an image later below.

---

## 3. Code (with Comments)
Paste your final Python program below. Include meaningful comments for each logical block: mapping, timing constants, encoding function, loop sending digits, cleanup.

---

## 4. Logic Explanation (Step-by-Step)
In clear sentences, briefly explain what the program does. Example outline:
1. Store Morse code patterns for digits 0–9.  
2. Ask user (or hardcode) for the student ID.  
3. For each digit, translate it into dots/dashes.  
4. Blink the LED: short ON for dot, long ON for dash.  
5. Use delays between symbols, digits, and full repetitions.  
6. Repeat or end; release the GPIO resources.

Keep it concise (5–8 sentences max).

---

## 5. Output (Run Evidence)
Provide BOTH software and physical evidence.

### 5.1 Terminal Output
Describe or paste the actual console text (e.g., printing each digit before blinking). If none, explain what you observed instead.

### 5.2 Photo of Physical Circuit
Insert an image (drag & drop into notebook) showing:  
- Raspberry Pi GPIO wiring  
- Resistor + LED orientation  
- Clear view of the used pin row

> Label the photo (e.g., Figure 1 – Lab 1 Circuit).

### 5.3 (Optional) Short Video / GIF
If allowed, you may note a video filename stored in the repo.

---

## 6. Reflection
Answer briefly (bullet points fine):
- What did you learn about Python (loops, functions, timing, GPIO usage)?  
- What was easiest?  
- What was hardest or most confusing?  
- One improvement you would add with more time (e.g., user input for speed, repeating message, second LED, buzzer, error handling).  
- Any issue you troubleshot (and how you fixed it).

---

## 7. In-Class Demonstration Checklist
Confirm you can show the TA/instructor:
- [ ] Python script runs without errors.  
- [ ] LED blinks full student ID in Morse code correctly.  
- [ ] Proper timing: dot < dash; spacing between digits visible.  
- [ ] Code commented and readable.  
- [ ] GPIO cleaned up (no warning messages).  

> Have your terminal and physical setup ready before they come to you.

---

## Appendix (Optional)
Add any extra experiments, timing calculations, or alternate versions of the code here.

---

End of Report.

In [ ]:
"""Lab 1 – Morse Code LED
Fill in group info, adjust pin number & student ID.
Replace placeholder comments with your explanations.
"""

# ========= 1. Imports & Setup =========
import time
try:
    import RPi.GPIO as GPIO  # Raspberry Pi specific
except ModuleNotFoundError:
    # Allows editing/testing syntax on non-Pi machine without crashing.
    class MockGPIO:
        BCM = BOARD = OUT = None
        def setmode(self, *a, **k): pass
        def setup(self, *a, **k): pass
        def output(self, *a, **k): pass
        def cleanup(self): pass
    GPIO = MockGPIO()

# Choose numbering mode: BCM (GPIO numbers) or BOARD (physical pin numbers)
GPIO.setmode(GPIO.BCM)  # Change to GPIO.BOARD if you prefer physical numbering

LED_PIN = 17  # TODO: set to the pin you actually used
GPIO.setup(LED_PIN, GPIO.OUT)

# ========= 2. Timing Constants (seconds) =========
DOT_TIME = 0.2      # short blink duration (dot)
DASH_TIME = DOT_TIME * 3
INTRA_SYMBOL_GAP = DOT_TIME      # between symbols within a digit
INTER_DIGIT_GAP = DOT_TIME * 3   # gap between digits
INTER_MESSAGE_GAP = DOT_TIME * 7 # gap after full ID (if repeating)

# ========= 3. Morse Mapping for Digits =========
# Using '.' for dot, '-' for dash
MORSE_DIGITS = {
    '0': '-----',
    '1': '.----',
    '2': '..---',
    '3': '...--',
    '4': '....-',
    '5': '.....',
    '6': '-....',
    '7': '--...',
    '8': '---..',
    '9': '----.'
}

# ========= 4. Helper Functions =========

def blink_symbol(symbol: str):
    """Blink one Morse symbol (dot or dash)."""
    if symbol == '.':
        GPIO.output(LED_PIN, True)
        time.sleep(DOT_TIME)
    else:  # dash
        GPIO.output(LED_PIN, True)
        time.sleep(DASH_TIME)
    GPIO.output(LED_PIN, False)
    time.sleep(INTRA_SYMBOL_GAP)


def blink_digit(d: str):
    """Blink all symbols for a single digit, with gap at end."""
    pattern = MORSE_DIGITS.get(d)
    if pattern is None:
        print(f"Skipping invalid character: {d}")
        return
    print(f"Digit {d} -> {pattern}")  # terminal feedback
    for i, sym in enumerate(pattern):
        blink_symbol(sym)
    time.sleep(INTER_DIGIT_GAP - INTRA_SYMBOL_GAP)  # adjust because last symbol already added gap


def blink_id(student_id: str):
    """Blink each digit of the student ID once."""
    for d in student_id:
        blink_digit(d)

# ========= 5. Main Execution =========
if __name__ == "__main__":
    try:
        STUDENT_ID = "123456789"  # TODO: replace with your actual ID (digits only)
        # Optionally validate input
        if not STUDENT_ID.isdigit():
            raise ValueError("Student ID must contain only digits.")

        print("Starting Morse code blink for ID:", STUDENT_ID)
        blink_id(STUDENT_ID)
        print("Done.")
        # If you need it to repeat, wrap blink_id in a loop with a break condition
        # while True:
        #     blink_id(STUDENT_ID)
        #     time.sleep(INTER_MESSAGE_GAP)
    except KeyboardInterrupt:
        print("Interrupted by user.")
    finally:
        GPIO.cleanup()  # Always clean up
        print("GPIO cleaned up.")